In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
import pdfplumber
import re
import os
from dotenv import load_dotenv

# 환경변수 불러오기
load_dotenv()

#### File 불러오기

In [ ]:
# 파일 경로 지정
file_path = 'KIET_경제ㆍ산업_전망_2023-02(2)__2024년_경제산업_전망_13대_주력산업편.pdf'

#### PDF 파일 Page 별 Split

In [ ]:
# pdf 파일 페이지별 split 함수
def chunk_pdf_with_pdfplumber(file_path, start_page, end_page):
    chunks = []
    
    with pdfplumber.open(file_path) as pdf:
        # 주어진 페이지 범위에 해당하는 페이지들을 처리합니다.
        for page_num in range(start_page - 1, end_page):
            page = pdf.pages[page_num]
            text = page.extract_text()
            if text:
                # 이스케이프 문자와 '<숫자-숫자>' 형태의 문구 제거
                cleaned_text = re.sub(r'\n|\r|\t', ' ', text)  # 이스케이프 문자 제거
                cleaned_text = re.sub(r'표<\d+-\d+>', '', cleaned_text)  # '<숫자-숫자>' 형태 제거
                cleaned_text = re.sub(r'<(그림|표) \d+-\d+>', '', cleaned_text)  # '<그림 숫자-숫자>', '<표 숫자-숫자>' 형태 제거
                chunks.append(cleaned_text.strip())
                

    return chunks

# 특정 페이지 범위 설정 (예: 13페이지부터 뒤에서 4페이지까지)
file_path = "경제산업동향_2024_11_2r_(2).pdf"
start_page = 3
end_page = -2  # 뒤에서 4페이지까지 의미

# PDF 페이지 수를 파악해 범위를 계산
with pdfplumber.open(file_path) as pdf:
    total_pages = len(pdf.pages)
    if end_page < 0:
        end_page = total_pages + end_page + 1  # 뒤에서부터 페이지 수 계산

# split 한 페이지 텍스트 document 형식으로 변환
chunks = [Document(page_content=doc) for doc in chunk_pdf_with_pdfplumber(file_path, start_page, end_page)]

In [ ]:
chunks

#### Local FAISS에 Chunk 저장

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

# 벡터 db 생성
vectorstore = FAISS.from_documents(documents=chunks, embedding=OpenAIEmbeddings())

# 로컬에 db 저장
vectorstore.save_local('./db/faiss')